In [19]:
RandomForestCVModel('datasetIPPD.pickle')

Dataset size read: train 256 and test 65 

# Tuning hyper-parameters 

Starting grdsurch.fit(X_train, y_train)

Best parameters set found on development set:

{'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0}
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=2,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=6000, n_jobs=-1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)



{'datasetIPPD.pickle': {'model': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
             max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=2,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=6000, n_jobs=-1, oob_score=True, random_state=None,
             verbose=0, warm_start=False),
  'test_mean_y_4_comparing': 1532.3461538461536,
  'test_rmse_4_reporting': 341.99694315814094,
  'train_rmse_cv_4_picking': 420.57121146971787}}

# M4. Ensemble Reg. - Averaging - Random Forest  (CV score)

In [7]:
from sklearn.metrics import mean_squared_error 
import numpy as np
def rmse_scorer(model, X, y): 
    y_predict = model.predict(X)
    k = np.sqrt(mean_squared_error(y, y_predict))
    return k

In [18]:
def RandomForestCVModel(filename):
    #open file and get the dictionary
    import pickle
    from sklearn import preprocessing
    from sklearn.ensemble.forest import RandomForestRegressor
    from numpy.random import RandomState
    from sklearn.model_selection import GridSearchCV

    with open(filename, 'rb') as handle:
        data = pickle.load(handle)

    #extract X_train, y_train, X_test, t_test
    X_train = data['X_train']
    y_train = data['y_train']
    X_test = data['X_test']
    y_test = data['y_test']
    print("Dataset size read: train %d and test %d \n" %(len(y_train), len(y_test)))
    
    #Normalize
    X_train = preprocessing.normalize(X_train, norm='l1')
    X_test  = preprocessing.normalize(X_test,  norm='l1')
    
    ##############################################################
    tuned_parameters = []
    tuned_parameters.append( { 
                              "min_samples_split": [2],
                              "min_samples_leaf": [1,2],
                              "min_weight_fraction_leaf": [0.0],
                              "max_features": ["auto"],
                            })
    
    ##############################################################
    
    print("# Tuning hyper-parameters ")
    print()

    grdsurch = GridSearchCV(RandomForestRegressor(n_estimators=6000, criterion='mse', max_depth=None, 
                                                  min_samples_split=2, 
                                                  min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                                                  max_leaf_nodes=None, min_impurity_split=1e-07, 
                                                  bootstrap=True, oob_score=True, 
                                                  n_jobs=-1, random_state=None, verbose=0, warm_start=False), 
                       tuned_parameters, 
                       cv=3, 
                       n_jobs=-1, 
                       scoring=rmse_scorer)
    print('Starting grdsurch.fit(X_train, y_train)')
    
    grdsurch.fit(X_train, y_train)

    print("\nBest parameters set found on development set:")
    print()
    print(grdsurch.best_params_)
    
    print(grdsurch.best_estimator_)
    print()
    rmse_cv = grdsurch.best_score_

    #Reporting Score on Test Set
    model               = grdsurch.best_estimator_
    reporting_testscore = rmse_scorer(model, X_test, y_test)
    
    return {filename: {'train_rmse_cv_4_picking': rmse_cv, 
                       'test_rmse_4_reporting': reporting_testscore, 
                       'test_mean_y_4_comparing': y_test.mean(),
                       'model': model
                      }}